In [1]:
:load  ParseLoc

### create Location for test

In [2]:
x = Location {database = "ushvr12", dbdesc = "dev", host = "odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com", loc_name = "sedhp", loc_class = "oracle", loc_directory = "/u01/app/oracle/product/19.3.0.0/dbee_1", loc_remote_node = "ora-mke2-scanp.am.health.ge.com", loc_remote_login = "gehc_hvr5", loc_remote_pwd = "", loc_remote_port = 4344, loc_db_name = "", loc_db_user = "PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes", loc_description = "scdhp connection"}

####  Visualize Location

In [3]:
import Data.Text.Prettyprint.Doc

pretty x

Location {
  database = ushvr12
  dbdesc = dev
  host = odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
  loc_name = sedhp
  loc_class = oracle
  loc_directory = /u01/app/oracle/product/19.3.0.0/dbee_1
  loc_remote_node = ora-mke2-scanp.am.health.ge.com
  loc_remote_login = gehc_hvr5
  loc_remote_pwd = 
  loc_remote_port = 4344
  loc_db_name = 
  loc_db_user = PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes
  loc_description = scdhp connection
}

### load Json

In [4]:
import Data.Aeson
import qualified Data.ByteString.Lazy as B
import Control.Applicative

input <- B.readFile "data2.json"
let mm = decode input :: Maybe [Location]
(length.unjust) mm

2596

In [5]:
l = unjust mm
let l_example =  (l!!2)

In [6]:
pretty l_example

Location {
  database = ushvr12
  dbdesc = dev
  host = odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
  loc_name = sedhp
  loc_class = oracle
  loc_directory = /u01/app/oracle/product/19.3.0.0/dbee_1
  loc_remote_node = ora-mke2-scanp.am.health.ge.com
  loc_remote_login = gehc_hvr5
  loc_remote_pwd = 
  loc_remote_port = 4344
  loc_db_name = 
  loc_db_user = PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes
  loc_description = scdhp connection
}

----------------

In [ ]:
import qualified Data.Text as T (Text,append,pack,unpack,head,take,splitOn,intercalate)
import Data.Maybe (fromMaybe)
import qualified Data.ByteString.Lazy.Char8 as BSL

--- Redshift parser
parseRedshift :: Location  -> [T.Text]
parseRedshift l  = getUSer l : (map T.pack . concat . filterCols . fromMaybe []. parseJson .  getJsonStr)  l  
   where
       getJsonStr  = BSL.pack. T.unpack .last . T.splitOn "=" . loc_db_name  -- get loc_db_name and return string after first '='
       parseJson s = decode s :: Maybe [[String]]            -- parse to list
       filterCols  = map tail .  filter (\[k, _] -> k `elem` ["db_node", "db_name"])
       getUSer     = head . T.splitOn "/" . loc_db_user
       
--- PARSER 

parseLoc :: Location  -> Location'
parseLoc l 
    | loc_class l == "file" && ((T.head . loc_directory)   l  == '/' )     = file_locdir l
    | loc_class l == "file" && ((T.take 4 . loc_directory) l  == "s3s:" )  = file_s3s l
    | loc_class l == "file" && ((T.take 5 . loc_directory) l  == "sftp:")  = file_sftp l
    | loc_class l `elem` ["mysql","greenplum","postgresql"]                = mysql l 
    | loc_class l == "sqlserver"                                           = sqlserver l
    | loc_class l == "teradata"                                            = teradata l 
    | loc_class l == "salesforce"                                          = salesforce l 
    | loc_class l == "redshift"                                            = redsfhit l  
    | otherwise   = loc2Location l ""  ["###","###","###"]                                           
     where
        file_locdir x = loc2Location x  "_locdir"  [loc_remote_login  x ,loc_remote_node x, loc_directory x]
        --
        getBacket  =  head . T.splitOn "/"  . last. T.splitOn "@" .loc_directory
        getPrefix  =  T.intercalate  "/" .tail . T.splitOn "/" . last. T.splitOn "@" .loc_directory
        file_s3s x =  loc2Location x "_s3s" ["###",getBacket x ,getPrefix x]
        --
        getsftpNode   = getBacket
        getsftpFolder =  getPrefix
        file_sftp x   = loc2Location x "_sftp" ["###",getsftpNode x ,getsftpFolder x]
        --
        getNodeMysql  =  head . T.splitOn "~" . loc_db_name
        getInstMysql  =  last . T.splitOn "~" . loc_db_name
        getUserMysql  =  head . T.splitOn "/" . loc_db_user
        mysql x =  loc2Location x ""   [ getUserMysql x , getNodeMysql x, getInstMysql x]
        --
        getNodeMsSql  =  head . T.splitOn "\\" . loc_db_name
        getInstMsSql  =  last . T.splitOn "\\" . loc_db_name
        getUserMsSql  =  getUserMysql
        sqlserver x   = loc2Location x ""   [getUserMsSql x, getNodeMsSql x,getInstMsSql x]
        --
        getUserTd     =  getUserMysql
        teradata x = loc2Location x ""   [getUserTd x, loc_db_name x,getUserTd x]
        --
        getNodeSf     = last .T.splitOn "//" . loc_directory
        getInstSf     = last .T.splitOn "@" . head . T.splitOn "/" . loc_db_user
        getUserSf     = head .T.splitOn "@" . head . T.splitOn "/" . loc_db_user
        salesforce x  = loc2Location x ""   [getUserSf x, getNodeSf x,getInstSf x]
        --
        redsfhit x    = loc2Location x ""  (parseRedshift x)

In [15]:
length ll

138

In [ ]:
ff l = loc_class l == "file" && ((T.take 5 . loc_directory) l  == "sftp:")  
f l  = loc_class l `elem` ["redshift"]
ll   = filter  f   l

Line 2: Use ==
Found:
loc_class l `elem` ["redshift"]
Why not:
loc_class l == "redshift"

In [11]:


r = head  ll
getJsonStr  =  last . T.splitOn "=" . loc_db_name
json_Str =  getJsonStr  r

-- Assuming loc_db_name and r are defined somewhere

:t json_Str

json_Str

json_Str :: Text

"[[\"odbc_driver\",\"/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so\"],[\"db_node\",\"us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com\"],[\"db_port\",\"5439\"],[\"db_name\",\"usinnovationredshift\"]]"

In [35]:
import qualified Data.ByteString.Char8 as B
(B.pack . unpack) json_Str 

Line 2: Redundant bracket
Found:
(B.pack)
Why not:
B.pack

: 

In [19]:
import qualified Data.ByteString.Char8 as BS


In [45]:
import Data.Maybe (fromMaybe )
getJsonStr  = BSL.pack. T.unpack .last . T.splitOn "=" . loc_db_name  -- get loc_db_name and return string after first '='
parseJson s = decode s :: Maybe [[String]]            -- parse to list
filterCols  = map tail .  filter (\[k, _] -> k `elem` ["db_node", "db_name"])
getUSer     = head . T.splitOn "/" . loc_db_user

getUSer r  : (map T.pack . concat .filterCols.fromMaybe [].parseJson . getJsonStr) r

["504005930","us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com","usinnovationredshift"]

In [28]:
import qualified Data.ByteString.Lazy.Char8 as BSL
import qualified Data.Text as T (Text,append,pack,unpack,head,take,splitOn,intercalate)

sText =  T.pack "[[\"odbc_driver\",\"/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so\"],[\"db_node\",\"us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com\"],[\"db_port\",\"5439\"],[\"db_name\",\"usinnovationredshift\"]]"
sString = T.unpack sText
bString = BSL.pack sString
decode bString :: Maybe Value

Just (Array [Array [String "odbc_driver",String "/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so"],Array [String "db_node",String "us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com"],Array [String "db_port",String "5439"],Array [String "db_name",String "usinnovationredshift"]])

In [50]:
import Data.Maybe (fromMaybe)
import qualified Data.Text as T (Text,append,pack,head,take,splitOn,intercalate,dropWhile )
import qualified Data.ByteString.Lazy.Char8 as BSL

parseRedshift :: Location  ->[T.Text]
parseRedshift l  = getUSer l : (map T.pack . concat . filterCols . fromMaybe []. parseJson .  getJsonStr)  l  
   where
       getJsonStr  = BSL.pack. T.unpack .last . T.splitOn "=" . loc_db_name  -- get loc_db_name and return string after first '='
       parseJson s = decode s :: Maybe [[String]]            -- parse to list
       filterCols  = map tail .  filter (\[k, _] -> k `elem` ["db_node", "db_name"])
       getUSer     = head . T.splitOn "/" . loc_db_user
       
parseRedshift r       

["504005930","us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com","usinnovationredshift"]

In [ ]:
(map (pretty.parseLoc) . take 5 ) ll

[Location {
  database         = ushvr12
  dbdesc           = dev
  host             = odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
  loc_class        = redshift
   loc_name         = tredp
   loc_directory    = 
   loc_remote_node  = 10.242.112.153
   loc_remote_login = isc_dev
   loc_remote_port  = 8443
   loc_db_user      = 504005930
   loc_description  = tredp
   db_node_name     = us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com
   db_instance      = usinnovationredshift
},Location {
  database         = ushvr00
  dbdesc           = fin-prod
  host             = odp-fin-prod-hvr-metadata.odp.health.ge.com
  loc_class        = redshift
   loc_name         = trdfp
   loc_directory    = 
   loc_remote_node  = 10.242.109.196
   loc_remote_login = hvr
   loc_remote_port  = 4343
   loc_db_user      = 504004225
   loc_description  = redshift finance prod
   db_node_name     = odp-fin-prod-etl-redshift.odp.health.ge.com
   db_instance      = g

In [50]:
import Data.List (nub)
(nub. map loc_class) l

["redshift","file","oracle","sqlserver","teradata","postgresql","salesforce","mysql","greenplum","kafka"]